In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from torchtext import data
from torchtext.data import TabularDataset
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
chart = train[['text','target']]
t_chart = test[['text']]

In [4]:
def remove(text):
    text = text.str.lower()
    text = text.str.replace('http\S+','')# http://와 https:// 두 가지 버전 있음
    text = text.str.replace('[0-9]','')# 사람 숫자, 날짜
    text = text.str.replace('@\S+','')# '@' 뒤에는 아이디 태그
    text = text.str.replace(',','')
    return text

In [5]:
chart["text"] = remove(chart["text"])
t_chart["text"] = remove(t_chart["text"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
chart.to_csv('train_r.csv',index = False)
t_chart.to_csv('test_r.csv',index = False)

In [7]:
pd.read_csv('train_r.csv').target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(pd.read_csv('train_r.csv'))

7613

# CSV -> trainset (= TabularDataset)

In [9]:
BATCH_SIZE = 64
lr = 0.001
epochs = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [10]:
TEXT = data.Field(sequential = True, tokenize = str.split, batch_first = True)
LABEL = data.LabelField(dtype = torch.long)

In [11]:
trainset = TabularDataset(path = 'train_r.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)])
testset = TabularDataset(path = 'test_r.csv', format='csv', fields=[('text', TEXT)])

In [12]:
trainset.examples[0].text, trainset.examples[0].label

(['text'], 'target')

In [13]:
trainset.examples.pop(0) # column을 제거해야 합니다.

In [14]:
trainset.examples[0].text, trainset.examples[0].label

(['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  '#earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 '1')

# trainset -> train_iter (= torchtext.data.Iterator)

In [15]:
TEXT.build_vocab(trainset, min_freq = 2)# 최소 2번 이상 등장한 단어만 사전에 담음
LABEL.build_vocab(trainset)

In [16]:
trainset, valset = trainset.split(split_ratio = 0.9)

In [17]:
train_iter = data.Iterator(dataset = trainset, batch_size = BATCH_SIZE, device = DEVICE)
val_iter = data.Iterator(dataset = valset, batch_size = BATCH_SIZE, device = DEVICE)
test_iter = data.Iterator(dataset = testset, batch_size = BATCH_SIZE, device = DEVICE)

In [18]:
vocab_size = len(TEXT.vocab)
n_classes = 2

# model

In [19]:
class LSTM_model(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embedding_dim, n_classes):
        super(LSTM_model, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embedding_dim)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        # x = [64, 27]
        x = self.embed(x)
        # x = [64, 27, 128]
        h_0 = self._init_state(batch_size = x.size(0))#첫 번째 은닉 벡터 정의
        # h_0 = [1, 64, 256]
        x, _ = self.lstm(x,(h_0,h_0))
        # x = [64, 27, 256]
        h_t = x[:,-1,:]
        # h_t = [64, 256]
        logit = self.out(h_t)
        # logit = [64, 2]
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

# Train model

In [20]:
def train(model, optimizer, train_iter):
    model.train()
    acc, total_loss = 0, 0
    for b,batch in tqdm(enumerate(train_iter)):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()# 기울기 0으로 초기화
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'mean')
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        loss.backward()
        optimizer.step()
    size = len(train_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

# Evaluate model

In [21]:
def evaluate(model, val_iter):
    model.eval()
    acc, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum')#오차의 합 구하고 total_loss에 더해줌
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

In [22]:
from tqdm import tqdm

# Train

In [28]:
model = LSTM_model(1, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [29]:
model2 = LSTM_model(2, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [30]:
model3 = LSTM_model(3, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [26]:
from time import sleep

In [31]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  5.18it/s]


2it [00:00,  4.89it/s]


3it [00:00,  4.88it/s]


4it [00:00,  4.97it/s]


5it [00:01,  4.87it/s]


6it [00:01,  4.55it/s]


7it [00:01,  3.92it/s]


8it [00:01,  3.82it/s]


9it [00:02,  3.89it/s]


10it [00:02,  3.73it/s]


11it [00:02,  3.49it/s]


12it [00:03,  3.30it/s]


13it [00:03,  3.20it/s]


14it [00:03,  3.13it/s]


15it [00:04,  3.06it/s]


16it [00:04,  3.05it/s]


17it [00:04,  2.87it/s]


18it [00:05,  2.77it/s]


19it [00:05,  2.81it/s]


20it [00:05,  2.96it/s]


21it [00:06,  2.98it/s]


22it [00:06,  3.00it/s]


23it [00:06,  3.05it/s]


24it [00:07,  3.13it/s]


25it [00:07,  3.29it/s]


26it [00:07,  3.36it/s]


27it [00:08,  3.34it/s]


28it [00:08,  3.40it/s]


29it [00:08,  3.68it/s]


30it [00:08,  3.76it/s]


31it [00:08,  3.92it/s]


32it [00:09,  3.88it/s]


33it [00:09,  3.99it/s]


34it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>




  5%|████▏                                                                              | 1/20 [00:27<08:36, 27.18s/it]


0it [00:00, ?it/s]


1it [00:00,  5.44it/s]


2it [00:00,  5.68it/s]


3it [00:00,  5.72it/s]


4it [00:00,  5.42it/s]


5it [00:00,  5.30it/s]


6it [00:01,  5.35it/s]


7it [00:01,  5.28it/s]


8it [00:01,  5.18it/s]


9it [00:01,  5.12it/s]


10it [00:01,  5.02it/s]


11it [00:02,  4.94it/s]


12it [00:02,  4.99it/s]


13it [00:02,  4.99it/s]


14it [00:02,  5.23it/s]


15it [00:02,  5.19it/s]


16it [00:03,  5.15it/s]


17it [00:03,  5.18it/s]


18it [00:03,  5.08it/s]


19it [00:03,  4.95it/s]


20it [00:03,  4.82it/s]


21it [00:04,  5.03it/s]


22it [00:04,  5.00it/s]


23it [00:04,  5.04it/s]


24it [00:04,  5.08it/s]


25it [00:04,  4.95it/s]


26it [00:05,  4.98it/s]


27it [00:05,  5.08it/s]


28it [00:05,  5.25it/s]


29it [00:05,  5.17it/s]


30it [00:05,  5.15it/s]


31it [00:06,  5.21it/s]


32it [00:06,  5.08it/s]


33it [00:06,  5.09it/s]


34it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.70>> <<val_accuracy : 54.00>>




 10%|████████▎                                                                          | 2/20 [00:49<07:43, 25.74s/it]


0it [00:00, ?it/s]


1it [00:00,  5.19it/s]


2it [00:00,  5.18it/s]


3it [00:00,  5.03it/s]


4it [00:00,  4.98it/s]


5it [00:00,  5.05it/s]


6it [00:01,  4.94it/s]


7it [00:01,  4.93it/s]


8it [00:01,  5.04it/s]


9it [00:01,  5.01it/s]


10it [00:01,  5.10it/s]


11it [00:02,  5.04it/s]


12it [00:02,  4.99it/s]


13it [00:02,  4.95it/s]


14it [00:02,  4.90it/s]


15it [00:03,  4.91it/s]


16it [00:03,  4.94it/s]


17it [00:03,  4.94it/s]


18it [00:03,  4.78it/s]


19it [00:03,  4.82it/s]


20it [00:04,  4.75it/s]


21it [00:04,  4.58it/s]


22it [00:04,  4.55it/s]


23it [00:04,  4.61it/s]


24it [00:04,  4.68it/s]


25it [00:05,  4.64it/s]


26it [00:05,  4.63it/s]


27it [00:05,  4.75it/s]


28it [00:05,  4.56it/s]


29it [00:06,  4.53it/s]


30it [00:06,  4.59it/s]


31it [00:06,  4.63it/s]


32it [00:06,  4.78it/s]


33it [00:06,  4.71it/s]


34it [

...



 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [07:55<00:56, 28.23s/it]


0it [00:00, ?it/s]


1it [00:00,  3.76it/s]


2it [00:00,  3.93it/s]


3it [00:00,  4.03it/s]


4it [00:00,  3.97it/s]


5it [00:01,  4.11it/s]


6it [00:01,  4.22it/s]


7it [00:01,  4.37it/s]


8it [00:01,  4.34it/s]


9it [00:02,  4.36it/s]


10it [00:02,  4.24it/s]


11it [00:02,  4.37it/s]


12it [00:02,  4.23it/s]


13it [00:03,  3.99it/s]


14it [00:03,  4.05it/s]


15it [00:03,  4.08it/s]


16it [00:03,  4.25it/s]


17it [00:04,  4.13it/s]


18it [00:04,  4.13it/s]


19it [00:04,  4.12it/s]


20it [00:04,  4.39it/s]


21it [00:04,  4.26it/s]


22it [00:05,  4.07it/s]


23it [00:05,  4.18it/s]


24it [00:05,  4.16it/s]


25it [00:05,  4.21it/s]


26it [00:06,  4.40it/s]


27it [00:06,  4.49it/s]


28it [00:06,  4.53it/s]


29it [00:06,  4.45it/s]


30it [00:07,  4.61it/s]


31it [00:07,  4.72it/s]


32it [00:07,  4.68it/s]


33it [00:07,  4.73it/s]


34it [

<<e : 19>> <<train_loss :  0.00>> <<train_accuracy : 98.00>> <<val_loss :  1.34>> <<val_accuracy : 75.00>>




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [08:19<00:27, 27.04s/it]


0it [00:00, ?it/s]


1it [00:00,  4.43it/s]


2it [00:00,  4.35it/s]


3it [00:00,  4.43it/s]


4it [00:00,  4.44it/s]


5it [00:01,  4.42it/s]


6it [00:01,  4.52it/s]


7it [00:01,  4.51it/s]


8it [00:01,  4.21it/s]


9it [00:02,  4.42it/s]


10it [00:02,  4.49it/s]


11it [00:02,  4.41it/s]


12it [00:02,  4.45it/s]


13it [00:02,  4.41it/s]


14it [00:03,  4.53it/s]


15it [00:03,  4.63it/s]


16it [00:03,  4.71it/s]


17it [00:03,  4.64it/s]


18it [00:04,  4.24it/s]


19it [00:04,  4.37it/s]


20it [00:04,  4.41it/s]


21it [00:04,  4.42it/s]


22it [00:04,  4.37it/s]


23it [00:05,  4.55it/s]


24it [00:05,  4.42it/s]


25it [00:05,  4.56it/s]


26it [00:05,  4.64it/s]


27it [00:06,  4.58it/s]


28it [00:06,  4.65it/s]


29it [00:06,  4.66it/s]


30it [00:06,  4.73it/s]


31it [00:06,  4.83it/s]


32it [00:07,  4.93it/s]


33it [00:07,  4.74it/s]


34it [

<<e : 20>> <<train_loss :  0.00>> <<train_accuracy : 98.00>> <<val_loss :  1.46>> <<val_accuracy : 74.00>>




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:44<00:00, 26.23s/it]


In [32]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model2, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model2, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  2.19it/s]


2it [00:00,  2.15it/s]


3it [00:01,  2.17it/s]


4it [00:01,  2.22it/s]


5it [00:02,  2.22it/s]


6it [00:02,  2.27it/s]


7it [00:03,  2.28it/s]


8it [00:03,  2.32it/s]


9it [00:03,  2.29it/s]


10it [00:04,  2.27it/s]


11it [00:04,  2.23it/s]


12it [00:05,  2.12it/s]


13it [00:05,  2.11it/s]


14it [00:06,  2.06it/s]


15it [00:06,  2.09it/s]


16it [00:07,  2.14it/s]


17it [00:07,  2.21it/s]


18it [00:08,  2.26it/s]


19it [00:08,  2.32it/s]


20it [00:08,  2.33it/s]


21it [00:09,  2.28it/s]


22it [00:09,  2.31it/s]


23it [00:10,  2.29it/s]


24it [00:10,  2.25it/s]


25it [00:11,  2.25it/s]


26it [00:11,  2.32it/s]


27it [00:12,  2.17it/s]


28it [00:12,  2.14it/s]


29it [00:13,  2.08it/s]


30it [00:13,  2.07it/s]


31it [00:14,  1.83it/s]


32it [00:14,  1.76it/s]


33it [00:15,  1.61it/s]


34it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.69>> <<val_accuracy : 45.00>>




  5%|████▏                                                                              | 1/20 [01:00<19:02, 60.11s/it]


0it [00:00, ?it/s]


1it [00:00,  2.44it/s]


2it [00:00,  2.51it/s]


3it [00:01,  2.45it/s]


4it [00:01,  2.42it/s]


5it [00:01,  2.54it/s]


6it [00:02,  2.63it/s]


7it [00:02,  2.63it/s]


8it [00:03,  2.61it/s]


9it [00:03,  2.52it/s]


10it [00:03,  2.50it/s]


11it [00:04,  2.54it/s]


12it [00:04,  2.52it/s]


13it [00:05,  2.55it/s]


14it [00:05,  2.54it/s]


15it [00:05,  2.54it/s]


16it [00:06,  2.58it/s]


17it [00:06,  2.55it/s]


18it [00:07,  2.55it/s]


19it [00:07,  2.43it/s]


20it [00:07,  2.40it/s]


21it [00:08,  2.42it/s]


22it [00:08,  2.50it/s]


23it [00:09,  2.44it/s]


24it [00:09,  2.52it/s]


25it [00:09,  2.57it/s]


26it [00:10,  2.66it/s]


27it [00:10,  2.65it/s]


28it [00:10,  2.72it/s]


29it [00:11,  2.71it/s]


30it [00:11,  2.57it/s]


31it [00:12,  2.55it/s]


32it [00:12,  2.50it/s]


33it [00:13,  2.48it/s]


34it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.69>> <<val_accuracy : 45.00>>




 10%|████████▎                                                                          | 2/20 [01:46<16:47, 56.00s/it]


0it [00:00, ?it/s]


1it [00:00,  2.40it/s]


2it [00:00,  2.29it/s]


3it [00:01,  2.14it/s]


4it [00:01,  2.10it/s]


5it [00:02,  2.08it/s]


6it [00:02,  2.12it/s]


7it [00:03,  2.15it/s]


8it [00:03,  2.12it/s]


9it [00:04,  2.10it/s]


10it [00:04,  2.07it/s]


11it [00:05,  2.01it/s]


12it [00:05,  2.04it/s]


13it [00:06,  2.01it/s]


14it [00:06,  2.00it/s]


15it [00:07,  2.11it/s]


16it [00:07,  2.18it/s]


17it [00:08,  2.17it/s]


18it [00:08,  2.16it/s]


19it [00:09,  2.08it/s]


20it [00:09,  2.09it/s]


21it [00:10,  2.03it/s]


22it [00:10,  1.98it/s]


23it [00:11,  1.99it/s]


24it [00:11,  1.99it/s]


25it [00:12,  2.03it/s]


26it [00:12,  1.96it/s]


27it [00:13,  1.90it/s]


28it [00:13,  1.93it/s]


29it [00:14,  1.87it/s]


30it [00:14,  1.90it/s]


31it [00:15,  1.90it/s]


32it [00:15,  1.95it/s]


33it [00:16,  1.89it/s]


34it [

...



 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [14:57<01:37, 48.76s/it]


0it [00:00, ?it/s]


1it [00:00,  2.13it/s]


2it [00:01,  1.96it/s]


3it [00:01,  2.03it/s]


4it [00:01,  2.07it/s]


5it [00:02,  2.11it/s]


6it [00:02,  2.01it/s]


7it [00:03,  1.91it/s]


8it [00:04,  1.91it/s]


9it [00:04,  1.85it/s]


10it [00:05,  1.67it/s]


11it [00:06,  1.64it/s]


12it [00:06,  1.67it/s]


13it [00:07,  1.64it/s]


14it [00:07,  1.64it/s]


15it [00:08,  1.71it/s]


16it [00:08,  1.75it/s]


17it [00:09,  1.75it/s]


18it [00:10,  1.72it/s]


19it [00:10,  1.66it/s]


20it [00:11,  1.65it/s]


21it [00:11,  1.69it/s]


22it [00:12,  1.73it/s]


23it [00:13,  1.73it/s]


24it [00:13,  1.66it/s]


25it [00:14,  1.65it/s]


26it [00:14,  1.64it/s]


27it [00:15,  1.62it/s]


28it [00:16,  1.69it/s]


29it [00:16,  1.72it/s]


30it [00:17,  1.75it/s]


31it [00:17,  1.77it/s]


32it [00:18,  1.74it/s]


33it [00:18,  1.73it/s]


34it [

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.69>> <<val_accuracy : 45.00>>




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [15:51<00:50, 50.51s/it]


0it [00:00, ?it/s]


1it [00:00,  2.32it/s]


2it [00:00,  2.37it/s]


3it [00:01,  2.47it/s]


4it [00:01,  2.48it/s]


5it [00:02,  2.48it/s]


6it [00:02,  2.42it/s]


7it [00:02,  2.42it/s]


8it [00:03,  2.41it/s]


9it [00:03,  2.43it/s]


10it [00:04,  2.35it/s]


11it [00:04,  2.29it/s]


12it [00:04,  2.35it/s]


13it [00:05,  2.33it/s]


14it [00:05,  2.41it/s]


15it [00:06,  2.39it/s]


16it [00:06,  2.33it/s]


17it [00:07,  2.19it/s]


18it [00:07,  2.15it/s]


19it [00:08,  2.20it/s]


20it [00:08,  2.26it/s]


21it [00:08,  2.27it/s]


22it [00:09,  2.34it/s]


23it [00:09,  2.37it/s]


24it [00:10,  2.34it/s]


25it [00:10,  2.41it/s]


26it [00:11,  2.34it/s]


27it [00:11,  2.19it/s]


28it [00:12,  2.20it/s]


29it [00:12,  2.28it/s]


30it [00:12,  2.29it/s]


31it [00:13,  2.28it/s]


32it [00:13,  2.20it/s]


33it [00:14,  2.12it/s]


34it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.69>> <<val_accuracy : 45.00>>




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [16:48<00:00, 50.44s/it]


In [33]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model3, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model3, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  1.19it/s]


2it [00:01,  1.26it/s]


3it [00:02,  1.32it/s]


4it [00:02,  1.36it/s]


5it [00:03,  1.47it/s]


6it [00:03,  1.56it/s]


7it [00:04,  1.62it/s]


8it [00:05,  1.68it/s]


9it [00:05,  1.69it/s]


10it [00:06,  1.73it/s]


11it [00:06,  1.74it/s]


12it [00:07,  1.80it/s]


13it [00:07,  1.90it/s]


14it [00:08,  1.94it/s]


15it [00:08,  1.98it/s]


16it [00:09,  1.98it/s]


17it [00:09,  1.90it/s]


18it [00:10,  1.82it/s]


19it [00:11,  1.73it/s]


20it [00:11,  1.79it/s]


21it [00:12,  1.83it/s]


22it [00:12,  1.91it/s]


23it [00:13,  1.96it/s]


24it [00:13,  2.07it/s]


25it [00:13,  2.11it/s]


26it [00:14,  2.00it/s]


27it [00:14,  2.00it/s]


28it [00:15,  1.95it/s]


29it [00:16,  1.89it/s]


30it [00:16,  1.85it/s]


31it [00:17,  1.83it/s]


32it [00:17,  1.75it/s]


33it [00:18,  1.73it/s]


34it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>




  5%|████▏                                                                              | 1/20 [01:11<22:41, 71.67s/it]


0it [00:00, ?it/s]


1it [00:00,  1.33it/s]


2it [00:01,  1.32it/s]


3it [00:02,  1.36it/s]


4it [00:02,  1.34it/s]


5it [00:03,  1.37it/s]


6it [00:04,  1.38it/s]


7it [00:05,  1.38it/s]


8it [00:05,  1.41it/s]


9it [00:06,  1.43it/s]


10it [00:07,  1.45it/s]


11it [00:07,  1.49it/s]


12it [00:08,  1.52it/s]


13it [00:08,  1.56it/s]


14it [00:09,  1.61it/s]


15it [00:10,  1.58it/s]


16it [00:10,  1.56it/s]


17it [00:11,  1.65it/s]


18it [00:12,  1.63it/s]


19it [00:12,  1.65it/s]


20it [00:13,  1.65it/s]


21it [00:13,  1.63it/s]


22it [00:14,  1.69it/s]


23it [00:15,  1.66it/s]


24it [00:15,  1.69it/s]


25it [00:16,  1.73it/s]


26it [00:16,  1.78it/s]


27it [00:17,  1.69it/s]


28it [00:17,  1.67it/s]


29it [00:18,  1.74it/s]


30it [00:19,  1.72it/s]


31it [00:19,  1.79it/s]


32it [00:20,  1.77it/s]


33it [00:20,  1.80it/s]


34it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>




 10%|████████▎                                                                          | 2/20 [02:21<21:19, 71.06s/it]


0it [00:00, ?it/s]


1it [00:00,  1.42it/s]


2it [00:01,  1.43it/s]


3it [00:02,  1.44it/s]


4it [00:02,  1.44it/s]


5it [00:03,  1.38it/s]


6it [00:04,  1.40it/s]


7it [00:05,  1.38it/s]


8it [00:05,  1.28it/s]


9it [00:06,  1.26it/s]


10it [00:07,  1.23it/s]


11it [00:08,  1.27it/s]


12it [00:09,  1.30it/s]


13it [00:09,  1.31it/s]


14it [00:10,  1.31it/s]


15it [00:11,  1.33it/s]


16it [00:11,  1.36it/s]


17it [00:12,  1.31it/s]


18it [00:13,  1.30it/s]


19it [00:14,  1.33it/s]


20it [00:14,  1.39it/s]


21it [00:15,  1.43it/s]


22it [00:16,  1.49it/s]


23it [00:16,  1.48it/s]


24it [00:17,  1.48it/s]


25it [00:18,  1.48it/s]


26it [00:18,  1.56it/s]


27it [00:19,  1.50it/s]


28it [00:20,  1.51it/s]


29it [00:20,  1.51it/s]


30it [00:21,  1.55it/s]


31it [00:22,  1.56it/s]


32it [00:22,  1.57it/s]


33it [00:23,  1.64it/s]


34it [

...



 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [18:32<02:00, 60.42s/it]


0it [00:00, ?it/s]


1it [00:00,  1.84it/s]


2it [00:01,  1.87it/s]


3it [00:01,  1.90it/s]


4it [00:02,  1.89it/s]


5it [00:02,  1.90it/s]


6it [00:03,  1.88it/s]


7it [00:03,  1.86it/s]


8it [00:04,  1.89it/s]


9it [00:04,  1.87it/s]


10it [00:05,  1.89it/s]


11it [00:05,  1.91it/s]


12it [00:06,  2.05it/s]


13it [00:06,  2.05it/s]


14it [00:07,  2.12it/s]


15it [00:07,  2.07it/s]


16it [00:08,  2.09it/s]


17it [00:08,  2.02it/s]


18it [00:09,  2.05it/s]


19it [00:09,  2.02it/s]


20it [00:10,  1.98it/s]


21it [00:10,  1.98it/s]


22it [00:11,  2.02it/s]


23it [00:11,  2.00it/s]


24it [00:12,  1.97it/s]


25it [00:12,  2.00it/s]


26it [00:13,  1.98it/s]


27it [00:13,  2.06it/s]


28it [00:14,  2.08it/s]


29it [00:14,  2.09it/s]


30it [00:15,  2.06it/s]


31it [00:15,  2.09it/s]


32it [00:16,  2.03it/s]


33it [00:16,  2.00it/s]


34it [

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>




 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [19:36<01:01, 61.53s/it]


0it [00:00, ?it/s]


1it [00:00,  2.05it/s]


2it [00:01,  1.97it/s]


3it [00:01,  1.96it/s]


4it [00:02,  1.96it/s]


5it [00:02,  1.84it/s]


6it [00:03,  1.85it/s]


7it [00:03,  1.81it/s]


8it [00:04,  1.80it/s]


9it [00:04,  1.79it/s]


10it [00:05,  1.75it/s]


11it [00:06,  1.75it/s]


12it [00:06,  1.79it/s]


13it [00:07,  1.77it/s]


14it [00:07,  1.81it/s]


15it [00:08,  1.88it/s]


16it [00:08,  1.86it/s]


17it [00:09,  1.88it/s]


18it [00:09,  1.80it/s]


19it [00:10,  1.82it/s]


20it [00:10,  1.81it/s]


21it [00:11,  1.84it/s]


22it [00:12,  1.91it/s]


23it [00:12,  1.87it/s]


24it [00:13,  1.85it/s]


25it [00:13,  1.89it/s]


26it [00:14,  1.98it/s]


27it [00:14,  2.03it/s]


28it [00:15,  2.01it/s]


29it [00:15,  1.92it/s]


30it [00:16,  1.97it/s]


31it [00:16,  2.00it/s]


32it [00:17,  1.97it/s]


33it [00:17,  2.03it/s]


34it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>




100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [20:35<00:00, 61.78s/it]
